In [1]:
import numpy as np
import numpy.typing as npt
import pandas as pd
from time import time

# Hiperparâmetros para geração das matrizes de teste

In [23]:
N_FACTORS = [1, 2, 5, 10]
idx_list = list[int]()
n_list = list[int]()
m_list = list[int]()
matrices_types = list[str]()
matrices = list[npt.NDArray]()
b_vecs = list[npt.NDArray]()
resid = list[float]()
method = list[str]()
times = list[float]()
execution_idx_list = list[int]()

curr_idx = 0
curr_execution_idx = 0
# Matrizes com proporção 5:3
for f in N_FACTORS:
    idx_list.append(curr_idx)
    curr_idx += 1
    n = 5 * f
    n_list.append(n)
    m = 3 * f
    m_list.append(m)
    matrices_types.append("5:3 Random")
    matrices.append(np.random.randint(1, 10, (n, m)))
    b_vecs.append(np.random.randint(1, 10, (n)))

# Matrizes rank deficient, proporção 5:3, com uma coluna sendo múltipla de outra.
for i, f in enumerate(N_FACTORS):
    idx_list.append(curr_idx)
    curr_idx += 1
    mat = matrices[i]
    n, m = mat.shape
    n_list.append(n)
    m_list.append(m)
    matrices_types.append("5:3 Random Rank Deficient")
    perm = np.random.permutation(m)
    mat[:, perm[0]] = np.random.randint(1, 10) * mat[:, perm[1]]
    matrices.append(mat)
    b_vecs.append(np.random.randint(0, 10, (n)))

In [24]:
matrices_df = pd.DataFrame(
    columns=["n", "m", "type"],
    data=zip(n_list, m_list, matrices_types),
    index=idx_list
)
matrices_df

,n,m,type
0,5,3,5:3 Random
1,10,6,5:3 Random
2,25,15,5:3 Random
3,50,30,5:3 Random
4,5,3,5:3 Random Rank Deficient
5,10,6,5:3 Random Rank Deficient
6,25,15,5:3 Random Rank Deficient
7,50,30,5:3 Random Rank Deficient


In [25]:
exec_mat_idx = list[int]()

# Normal equations

In [26]:
for (i, (idx, A, b)) in enumerate(zip(idx_list, matrices, b_vecs)):
    execution_idx_list.append(curr_execution_idx)
    curr_execution_idx += 1
    exec_mat_idx.append(idx)
    method.append("Normal Equations")
    start = time()
    B = A.T @ A
    y = A.T @ b
    G = np.linalg.cholesky(B)
    z = np.linalg.solve(G, y)
    x = np.linalg.solve(G.T, z)
    t_resid = np.linalg.norm(b - A @ x)
    resid.append(t_resid)
    end = time()
    times.append((end - start))

LinAlgError: Matrix is not positive definite

# QR decomposition

In [27]:
for (i, (idx, A, b)) in enumerate(zip(idx_list, matrices, b_vecs)):
    execution_idx_list.append(curr_execution_idx)
    curr_execution_idx += 1
    exec_mat_idx.append(idx)
    method.append("QR")
    start = time()
    Q, R = np.linalg.qr(matrices[i])
    c = Q.T @ b_vecs[i]
    x = np.linalg.solve(R, c)
    t_resid = np.linalg.norm(b_vecs[i] - matrices[i] @ x)
    resid.append(t_resid)
    end = time()
    times.append((end - start))

# SVD decompoosition

In [28]:
for i, n in enumerate(N_FACTORS):
    execution_idx_list.append(curr_execution_idx)
    curr_execution_idx += 1
    method.append("SVD")
    start = time()
    U, S, Vt = np.linalg.svd(matrices[i])
    z = U.T @ b_vecs[i]
    y = z.copy()[:S.shape[0]] / S
    x = Vt.T @ y
    t_resid = np.linalg.norm(b_vecs[i] - matrices[i] @ x)
    resid.append(t_resid)
    end = time()
    times.append((end - start))

In [29]:
times_ms = [t * 1000 for t in times]
pd.DataFrame(
    columns=["Method", "Residual", "Execution Time (ms)"],
    data=zip(method, resid, times_ms)
)

,Method,Residual,Execution Time (ms)
0,Normal Equations,2.208825,0.296116
1,QR,7.318641,0.164747
2,QR,7.924329,0.222683
3,QR,40.216968,0.632048
4,QR,9.107861,0.204086
5,QR,11.500000,0.226736
6,QR,8.051954,0.238180
7,QR,24.721232,0.833988
8,QR,6.247265,1.636505
9,SVD,27.838822,0.300407
